In [87]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

# Provide the path to the Chrome WebDriver executable
webdriver_path = '/path/to/chromedriver'

# Configure the ChromeDriver service
service = Service(webdriver_path)

# Configure ChromeOptions to run in headless mode
options = Options()
options.add_argument('--headless')

# Configure Selenium webdriver with the service and options
driver = webdriver.Chrome(service=service, options=options)

url = "https://www.twse.com.tw/zh/products/securities/etf/products/domestic.html"

# Open the URL in the browser
driver.get(url)

driver.implicitly_wait(2)

# Find the total number of rows in the table
rows = driver.find_elements(By.XPATH, '//*[@id="reports"]/div[1]/div[2]/table/tbody/tr')

href_list = []

# Iterate through each row in the table
for i in range(1, len(rows) + 1):
    # Construct the XPath expression for each row dynamically
    xpath_expression = '//*[@id="reports"]/div[1]/div[2]/table/tbody/tr[' + str(i) + ']/td[1]/a'
    
    # Find the element using the XPath expression
    element = driver.find_element(By.XPATH, xpath_expression)
    
    # Extract the href attribute
    href_attribute = element.get_attribute("href")
    
    # Append the href attribute to the list
    href_list.append(href_attribute)

# Close the browser
driver.quit()


In [88]:
def get_basic_data(url):
    from selenium import webdriver
    from selenium.webdriver.chrome.service import Service
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.common.by import By

    # Provide the path to the Chrome WebDriver executable
    webdriver_path = '/path/to/chromedriver'

    # Configure the ChromeDriver service
    service = Service(webdriver_path)

    # Configure ChromeOptions to run in headless mode
    options = Options()
    options.add_argument('--headless')

    # Configure Selenium webdriver with the service and options
    driver = webdriver.Chrome(service=service, options=options)

    # Open the URL in the browser
    driver.get(url)

    driver.implicitly_wait(2)

    # Find the tbody element using the XPath expression
    tbody_xpath = '//*[@id="table0"]/div[1]/div[2]/table/tbody'
    tbody_element = driver.find_element(By.XPATH, tbody_xpath)

    # Find all the td elements within the tbody
    td_elements = tbody_element.find_elements(By.TAG_NAME, "td")

    # Extract the text of each td element
    td_texts = [element.text for element in td_elements]

    # Close the browser
    driver.quit()

    return td_texts

In [89]:
data=[]
for i in range(len(href_list)):
    data.append(get_basic_data(href_list[i]))

In [ ]:
import re
string = "--H0: '名稱'; --H1: 'ETF簡稱'; --H2: '證券代號'; --H3: 'ETF類別'; --H4: '上市日期'; --H5: '基金經理公司'; --H6: '標的指數'; --H7: '追蹤方式'; --H8: '交易單位'; --H9: '交易價格'; --H10: '升降單位'; --H11: '升降幅度'; --H12: '交易時間'; --H13: '信用交易'; --H14: '證券交易稅'; --H15: '交易手續費費率'; --H16: '管理費'; --H17: '保管費'; --H18: '申購買回方式'; --H19: '申購買回申報時間'; --H20: '申購買回基本單位'; --H21: '收益分配'; --H22: '基金經理公司網站'; --W: 8.5em;"
columns = re.findall(r"--H\d+:\s*'(.*?)'", string)
basic = pd.DataFrame(data, columns=columns)
basic

In [144]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('etf.sqlite')

# Create a cursor object to execute SQL commands
cursor = conn.cursor()

# Get the list of tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cursor.fetchall()

# Iterate over the tables and retrieve their structure
for table in tables:
    table_name = table[0]
    print("Table:", table_name)
    
    # Get the columns of the table
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = cursor.fetchall()
    
    # Print the column details
    for column in columns:
        column_name = column[1]
        data_type = column[2]
        is_nullable = "NOT NULL" if column[3] == 0 else "NULL"
        is_primary_key = "PRIMARY KEY" if column[5] == 1 else ""
        
        print(f"- {column_name} ({data_type}) {is_nullable} {is_primary_key}")

    print()  # Print a blank line between tables

# Close the connection
conn.close()


Table: etfs
- 年季 (TEXT) NOT NULL 
- ETF名稱 (TEXT) NOT NULL 
- 股票代號 (TEXT) NOT NULL 
- 股票名稱 (TEXT) NOT NULL 
- 持股比率 (TEXT) NOT NULL 
- 產業類別 (TEXT) NOT NULL 
- 產業比率 (TEXT) NOT NULL 
- 代號簡稱 (TEXT) NOT NULL 

Table: basic
- 名稱 (TEXT) NOT NULL 
- ETF簡稱 (TEXT) NOT NULL 
- 證券代號 (TEXT) NOT NULL 
- ETF類別 (TEXT) NOT NULL 
- 上市日期 (TEXT) NOT NULL 
- 基金經理公司 (TEXT) NOT NULL 
- 標的指數 (TEXT) NOT NULL 
- 追蹤方式 (TEXT) NOT NULL 
- 交易單位 (TEXT) NOT NULL 
- 交易價格 (TEXT) NOT NULL 
- 升降單位 (TEXT) NOT NULL 
- 升降幅度 (TEXT) NOT NULL 
- 交易時間 (TEXT) NOT NULL 
- 信用交易 (TEXT) NOT NULL 
- 證券交易稅 (TEXT) NOT NULL 
- 交易手續費費率 (TEXT) NOT NULL 
- 管理費 (TEXT) NOT NULL 
- 保管費 (TEXT) NOT NULL 
- 申購買回方式 (TEXT) NOT NULL 
- 申購買回申報時間 (TEXT) NOT NULL 
- 申購買回基本單位 (TEXT) NOT NULL 
- 收益分配 (TEXT) NOT NULL 
- 基金經理公司網站 (TEXT) NOT NULL 



In [146]:
import pandas as pd
import sqlite3

# Assume you have a DataFrame named "basic"
# basic = ...

# Connect to the SQLite database
conn = sqlite3.connect('etf.sqlite')

# Save the DataFrame to the database as a new table
basic.to_sql('basic', conn, if_exists='replace', index=False)

# Close the connection
conn.close()


In [1]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('etf.sqlite')

# Query all data from the "etfs" table
query = "SELECT * FROM etfs"
df = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Print the DataFrame
df[(df['年季']=='11201') & (df['代號簡稱']=="00923 群益台灣ESG低碳")]

,年季,ETF名稱,股票代號,股票名稱,持股比率,產業類別,產業比率,代號簡稱
2072,11201,群益台灣ESG低碳50ETF證券投資信託基金,1216,統一,1.26%,食品工業,1.26%,00923 群益台灣ESG低碳
2073,11201,群益台灣ESG低碳50ETF證券投資信託基金,1402,遠東新,0.41%,紡織纖維,0.71%,00923 群益台灣ESG低碳
2074,11201,群益台灣ESG低碳50ETF證券投資信託基金,1476,儒鴻,0.30%,紡織纖維,0.71%,00923 群益台灣ESG低碳
2075,11201,群益台灣ESG低碳50ETF證券投資信託基金,1605,華新,0.61%,電器電纜,0.61%,00923 群益台灣ESG低碳
2076,11201,群益台灣ESG低碳50ETF證券投資信託基金,2609,陽明,0.58%,航運業,0.58%,00923 群益台灣ESG低碳
2077,11201,群益台灣ESG低碳50ETF證券投資信託基金,2834,臺企銀,0.31%,金融保險業,13.84%,00923 群益台灣ESG低碳
2078,11201,群益台灣ESG低碳50ETF證券投資信託基金,2880,華南金,0.81%,金融保險業,13.84%,00923 群益台灣ESG低碳
2079,11201,群益台灣ESG低碳50ETF證券投資信託基金,2881,富邦金,1.86%,金融保險業,13.84%,00923 群益台灣ESG低碳
2080,11201,群益台灣ESG低碳50ETF證券投資信託基金,2882,國泰金,1.49%,金融保險業,13.84%,00923 群益台灣ESG低碳
2081,11201,群益台灣ESG低碳50ETF證券投資信託基金,2883,開發金,0.70%,金融保險業,13.84%,00923 群益台灣ESG低碳


In [ ]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('etf.sqlite')

# Query the joined data from the tables
query = '''
SELECT *
FROM etfs
JOIN basic ON etfs."ETF名稱" = basic."名稱"
'''
df = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Print the joined DataFrame
df


In [ ]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('etf.sqlite')

# Query the joined data from the tables
query = '''
SELECT basic."證券代號" || ' ' || basic."ETF簡稱" AS "代號簡稱", etfs.*, basic.*
FROM etfs
JOIN basic ON etfs."ETF名稱" = basic."名稱"
'''
df = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Print the joined data
df


In [106]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('etf.sqlite')

# Query the joined data from the tables
query = '''
SELECT basic."證券代號" || ' ' || basic."ETF簡稱" AS "代號簡稱", etfs.*, basic.*
FROM etfs
JOIN basic ON etfs."ETF名稱" = basic."名稱"
'''
df = pd.read_sql_query(query, conn)

# Save the DataFrame to the database as a new table
df.to_sql('etfs_basic', conn, if_exists='replace', index=False)

# Close the connection
conn.close()


In [142]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('etf.sqlite')
c = conn.cursor()

# Drop the 代號簡稱 column from the etfs table if it exists
c.execute("PRAGMA table_info(etfs)")
columns = [column[1] for column in c.fetchall()]
if '代號簡稱' in columns:
    c.execute("ALTER TABLE etfs DROP COLUMN 代號簡稱")

# Remove the string from the basic table
c.execute("UPDATE basic SET 名稱 = REPLACE(名稱, '(本基金之配息來源可能為收益平準金)', '')")

# Add a new column to the etfs table
c.execute("ALTER TABLE etfs ADD COLUMN 代號簡稱 TEXT")

# Update the values in the new column based on the concatenation of 證券代號 and ETF簡稱 in basic table
c.execute("UPDATE etfs SET 代號簡稱 = (SELECT 證券代號 || ' ' || ETF簡稱 FROM basic WHERE basic.名稱 = etfs.ETF名稱)")

# Commit the changes and close the connection
conn.commit()
conn.close()


In [152]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('etf.sqlite')
c = conn.cursor()

# Drop the 代號簡稱 column from the etfs table if it exists
c.execute("PRAGMA table_info(etfs)")
columns = [column[1] for column in c.fetchall()]
if '代號簡稱' in columns:
    c.execute("ALTER TABLE etfs DROP COLUMN 代號簡稱")

# Remove the string from the basic table
c.execute("UPDATE basic SET 名稱 = REPLACE(名稱, '（本基金之配息來源可能為收益平準金）', '')")

# Add a new column to the etfs table
c.execute("ALTER TABLE etfs ADD COLUMN 代號簡稱 TEXT")

# Update the values in the new column based on the concatenation of 證券代號 and ETF簡稱 in basic table
c.execute('''UPDATE etfs SET 代號簡稱 = CASE
    WHEN ETF名稱 = '群益台灣ESG低碳50ETF證券投資信託基金' THEN '00923 群益台灣ESG低碳'
    ELSE (SELECT 證券代號 || ' ' || ETF簡稱 FROM basic WHERE basic.名稱 = etfs.ETF名稱)
    END''')

# Commit the changes and close the connection
conn.commit()
conn.close()
